In [ ]:
import cv2
import numpy as np

In [ ]:
# Wanted video to process
object = 'obj01'

# Read the input video
input_video = cv2.VideoCapture(f'./data/{object}.mp4')

# Get video properties
frame_width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = input_video.get(cv2.CAP_PROP_FPS)

# Create output video writer
output_video = cv2.VideoWriter(f'./output/{object}_mask.mp4', cv2.VideoWriter_fourcc(*'mp4'), fps, (frame_width, frame_height))

In [2]:
# Iterate through each frame of the input video
while True:
    ret, frame = input_video.read()

    if not ret: break

    # Apply seed-based watershed algorithm
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    _, markers = cv2.connectedComponents(binary)
    markers = markers + 1
    markers[markers == 2] = 0
    markers = cv2.watershed(frame, markers)

    # Convert markers to colored overlay
    overlay = np.zeros(frame.shape, dtype=np.uint8)
    overlay[markers == -1] = (0, 0, 255)  # Red color for boundaries

    # Apply the overlay on the frame
    result = cv2.addWeighted(frame, 1, overlay, 0.5, 0)

    # Write the transformed frame to the output video
    output_video.write(result)

# Release video objects
input_video.release()
output_video.release()
